In [1]:
print("he")

he


In [5]:
%pwd


'c:\\Users\\Asus\\Downloads\\End-to-end-Medical-Chatbot-Generative-AI-main\\research'

In [6]:
import os

os.chdir("../")

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
# exract data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf_file(data="Data/")

In [10]:
# split the data into text chunks


def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5859


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

### download this
- pip install -U langchain langchain-community sentence-transformers huggingface-hub


In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_14908\4025411284.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Asus\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("hello world")
print("length", len(query_result))

length 384


In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

### index created inside pinecone


In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '7595ccd5e3ab84aef04cddd81aff6007', 'date': 'Fri, 22 Aug 2025 05:13:52 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [19]:
# embed each chunk and upsert the embeddings into ur pinecode index
# it will create records in pinecone index(6k)
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

KeyboardInterrupt: 

In [20]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name, embedding=embeddings
)

In [21]:
docsearch

In [22]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [23]:
retrieved_docs = retriever.invoke("What is acne?")

In [24]:
retrieved_docs

[Document(id='119bd6e2-89e4-409f-bf11-ce1a11b9134c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='cef07560-5604-4626-8e8e-71bf6065366f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='30e330d0-7bdb-40f5-9fbb-253488392c2c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 

In [ ]:
%pip install langchain_google_genai

  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 11.8 MB/s eta 0:00:00
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)

   --- ------------------------------------  1/12 [pyasn1]
  Attempting uninstall: protobuf
   --- ------------------------------------  1/12 [pyasn1]
    Found existing installation: protobuf 5.29.5
   --- ------------------------------------  1/12 [pyasn1]
    Uninstalling protobuf-5.29.5:
   --- ------------------------------------  1/12 [pyasn1]
      Successfully uninstalled protobuf-5.29.5
   --- ------------------------------------  1/12 [pyasn1]
   ------ ---------------------------------  2/12 [protobuf]
   ------ ---------------------------------  2/12 [protobuf]
   ------ ---------------------------

  You can safely remove it manually.


In [ ]:
# from langchain_openai import OpenAI

# llm = OpenAI(temperature=0.4, max_tokens=500)

import os

GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

# ✅ Initialize Gemini LLM (replace with your Gemini API key)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # you can also use gemini-1.5-flash for cheaper/faster inference
    temperature=0.4,
    max_output_tokens=200,
    google_api_key=GEMINI_API_KEY,
)

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer or this document does not have the answer, say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [48]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland.  This leads to increased bone and soft tissue growth, along with other bodily disturbances.  The provided text does not fully define gigantism, but it is mentioned in relation to acromegaly.


In [49]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I'm sorry, but this document does not contain the answer to this question.  I do not have access to external websites or specific files online.
